In [214]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [215]:
# function to contruct back from tf record example
def prepare_sample(example):
    example["grid_map"] = tf.io.decode_raw(example["grid_map"], out_type=tf.float16)
    example["grid_map"] = tf.reshape(example["grid_map"],[1536,1536])
    print(example["grid_map"])

    example["grid_org_res"] = tf.io.decode_raw(example["grid_org_res"], out_type=tf.float32)
    #example["grid_org_res"] = tf.reshape(example["grid_org_res"],[1,3])

    example["left_bnd"] = tf.io.decode_raw(example["left_bnd"], out_type=tf.float32)
    example["left_bnd"] = tf.reshape(example["left_bnd"],[25,2])

    example["right_bnd"] = tf.io.decode_raw(example["right_bnd"], out_type=tf.float32)
    example["right_bnd"] = tf.reshape(example["right_bnd"],[25,2])

    example["car_odo"] = tf.io.decode_raw(example["car_odo"], out_type=tf.float32)
    #example["car_odo"] = tf.reshape(example["car_odo"],[1,3])

    example["init_path"] = tf.io.decode_raw(example["init_path"], out_type=tf.float32)
    example["init_path"] = tf.reshape(example["init_path"],[25,2])

    example["opt_path"] = tf.io.decode_raw(example["opt_path"], out_type=tf.float32)
    example["opt_path"] = tf.reshape(example["opt_path"],[25,2])
    return example

def parse_tfrecord_fn_ds(example):

    # Dict of feature and their description
    feature_description = {

        # model inputs
        "grid_map": tf.io.FixedLenFeature([], tf.string),
        "grid_org_res": tf.io.FixedLenFeature([], tf.string),
        "left_bnd": tf.io.FixedLenFeature([], tf.string),
        "right_bnd": tf.io.FixedLenFeature([], tf.string),
        "car_odo": tf.io.FixedLenFeature([], tf.string),
        "init_path": tf.io.FixedLenFeature([], tf.string),
        
        #model ouput
        "opt_path": tf.io.FixedLenFeature([], tf.string),

    }

    # Parse a single Example proto
    example = tf.io.parse_single_example(example, feature_description)
    return example

In [216]:
#grid_path = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/2021-06-10_14-25-32/2021-06-10_14-25-32_0_nos43_grid.npz'
#init_data_path = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/2021-06-10_14-25-32/2021-06-10_14-25-32_0_nos43_init_path.npz'
#opt_data_path = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/2021-06-10_14-25-32/2021-06-10_14-25-32_0_nos43_opt_path.npz'

## Load sample tfrecord (with costmap based grids)
tf_rec_smaple_path = '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records_w_costmap_dist_dir/scenario5/2021-06-10_14-25-32/2021-06-10_14-25-32_0_file_02-11.tfrec'
sample_raw_ds = tf.data.TFRecordDataset(tf_rec_smaple_path)
parsed_dataset = sample_raw_ds.map(parse_tfrecord_fn_ds).map(prepare_sample)
parsed_dataset.element_spec

Tensor("Reshape:0", shape=(1536, 1536), dtype=float16)


{'car_odo': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 'grid_map': TensorSpec(shape=(1536, 1536), dtype=tf.float16, name=None),
 'grid_org_res': TensorSpec(shape=(None,), dtype=tf.float32, name=None),
 'init_path': TensorSpec(shape=(25, 2), dtype=tf.float32, name=None),
 'left_bnd': TensorSpec(shape=(25, 2), dtype=tf.float32, name=None),
 'opt_path': TensorSpec(shape=(25, 2), dtype=tf.float32, name=None),
 'right_bnd': TensorSpec(shape=(25, 2), dtype=tf.float32, name=None)}

In [217]:
def plot_scene(features):    
    grid_map = features["grid_map"].numpy()
    grid_org = features["grid_org_res"].numpy()
    left_bnd = features["left_bnd"].numpy()
    right_bnd = features["right_bnd"].numpy()
    init_path = features["init_path"].numpy()
    opt_path = features["opt_path"].numpy()
    car_odo = features["car_odo"].numpy()

    #print(type(grid_map))
    plt.figure(figsize=(10, 10),dpi=200)
    #ax=fig.add_subplot(1,1,1)

    res = grid_org[2]
    plt.plot((left_bnd[:,0]-grid_org[0])/res,(left_bnd[:,1]-grid_org[1])/res,'-.', color='magenta',markersize=0.5, linewidth=0.5)

    plt.plot((init_path[:,0]-grid_org[0])/res,(init_path[:,1]-grid_org[1])/res,'o-', color='lawngreen',markersize=1, linewidth=1)
    plt.plot((opt_path[:,0]-grid_org[0])/res,(opt_path[:,1]-grid_org[1])/res,'--', color='yellow',markersize=1, linewidth=1)

    plt.plot((right_bnd[:,0]-grid_org[0])/res,(right_bnd[:,1]-grid_org[1])/res, '-.',color='magenta',markersize=0.5, linewidth=0.5)


    plt.plot((car_odo[0]-grid_org[0])/res,(car_odo[1]-grid_org[1])/res,'r*', color = 'red',markersize=8)


    plt.legend(['Left bound', 'gt_init_path', 'gt_opt_path','right bound', 'car_centre'], loc='lower left')

    plt.imshow(grid_map.astype(float),origin="lower",cmap='jet')
    plt.colorbar()

    return plt


In [218]:
data_pp={}
for features in parsed_dataset.skip(10).take(1):
    for key in features.keys():
        #if key != "grid_map":
        print(f"{key}: {np.shape(features[key].numpy())}")
        data_pp[key] = features[key]
        if key=='grid_map':
            print(f"{key}: {type(features[key])}")
    #_ = plot_scene(features=features)

car_odo: (3,)
grid_map: (1536, 1536)
grid_map: <class 'tensorflow.python.framework.ops.EagerTensor'>
grid_org_res: (3,)
init_path: (25, 2)
left_bnd: (25, 2)
opt_path: (25, 2)
right_bnd: (25, 2)


In [239]:
#plt.figure(figsize=(10, 10),dpi=100)
#plt.imshow(data_pp['grid_map'].numpy().astype(float),origin="lower",cmap='jet')
gridmap_idx=tf.where(data_pp["grid_map"] >0.35)
gridmap_idx = tf.cast(gridmap_idx,tf.float32)
print(data_pp["grid_map"].shape)

gridmap_idx

(1536, 1536)


<tf.Tensor: shape=(4311, 2), dtype=float32, numpy=
array([[ 580.,  797.],
       [ 580.,  798.],
       [ 581.,  799.],
       ...,
       [1089.,  764.],
       [1089.,  765.],
       [1089.,  766.]], dtype=float32)>

In [220]:
gridmap_idx_int = tf.cast(gridmap_idx,dtype=tf.int32)
cost_gather = tf.gather_nd(data_pp["grid_map"],gridmap_idx_int)
cost_gather

<tf.Tensor: shape=(4311,), dtype=float16, numpy=array([0.3784, 0.361 , 0.3718, ..., 0.3855, 0.3855, 0.3855], dtype=float16)>

In [221]:
gridmap_idx_int = tf.cast(gridmap_idx,dtype=tf.int32)
#cost_gridmap_idx = tf.e(shape=gridmap_idx_int.shape[0])
#data_pp['grid_map'][580,797]

costs_tensor = tf.convert_to_tensor([data_pp['grid_map'][gridmap_idx_int[i][0],gridmap_idx_int[i][1]] for i in range(gridmap_idx_int.shape[0])])
costs_tensor = tf.cast(costs_tensor,dtype=tf.float32)
costs_tensor

<tf.Tensor: shape=(4311,), dtype=float32, numpy=
array([0.37841797, 0.36108398, 0.37182617, ..., 0.38549805, 0.38549805,
       0.38549805], dtype=float32)>

In [223]:

tf_init_path =((data_pp["init_path"]-data_pp["grid_org_res"][:2])/data_pp["grid_org_res"][2])
tf_opt_path = ((data_pp["opt_path"]-data_pp["grid_org_res"][:2])/data_pp["grid_org_res"][2])
tf_grid_map = data_pp["grid_map"]
tf_init_path


<tf.Tensor: shape=(25, 2), dtype=float32, numpy=
array([[758.3725 , 595.9519 ],
       [754.65857, 608.59467],
       [751.3653 , 621.3199 ],
       [748.74347, 634.1666 ],
       [746.93445, 647.146  ],
       [745.99457, 660.248  ],
       [746.01245, 673.4674 ],
       [746.03217, 686.6876 ],
       [746.10535, 699.9163 ],
       [745.857  , 713.0991 ],
       [744.3494 , 726.15814],
       [742.813  , 739.01624],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ],
       [739.9198 , 751.6684 ]], dtype=float32)>

## calculate opt path cost with respect to grid

In [238]:
#(tf.square((gridmap_idx - tf_init_path[0])))

# for one point in path
#print(gridmap_idx,tf_init_path)
diff_sqr = K.square(gridmap_idx - tf_opt_path[5])
dist = K.sqrt(K.sum(diff_sqr, axis=1))
inv_dist = tf.math.reciprocal_no_nan(tf.math.pow(dist,0.1))
print(inv_dist,"\n",dist)

net_cost_for_point = tf.reduce_sum(tf.multiply(inv_dist,costs_tensor))
net_cost_for_point

tf.Tensor([0.58444273 0.5842698  0.58430403 ... 0.5553526  0.55530757 0.5552622 ], shape=(4311,), dtype=float32) 
 tf.Tensor([215.07048 215.7077  215.58138 ... 358.35345 358.64426 358.93762], shape=(4311,), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=1624.811>

## calculate init path cost with respect to grid

In [235]:
init_diff_sqr = K.square(gridmap_idx - tf_init_path[5])
init_dist = K.sqrt(K.sum(init_diff_sqr, axis=1))
init_inv_dist = tf.math.reciprocal_no_nan(tf.math.pow(init_dist,0.1))
print(init_inv_dist,"\n",init_dist)

init_net_cost_for_point = tf.reduce_sum(tf.multiply(init_inv_dist,costs_tensor))
init_net_cost_for_point

tf.Tensor([0.58444273 0.5842698  0.58430403 ... 0.5553526  0.55530757 0.5552622 ], shape=(4311,), dtype=float32) 
 tf.Tensor([215.07048 215.7077  215.58138 ... 358.35345 358.64426 358.93762], shape=(4311,), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=1624.811>

In [210]:
init_dist_0 = tf.sqrt(tf.reduce_sum(tf.square(gridmap_idx-tf_init_path[0]),axis=1))